In [76]:
import dask.array as da
import napari
import btrack
from tqdm.auto import tqdm
from skimage.io import imshow,  imsave
import matplotlib.pyplot as plt
import numpy as np
import itertools
import json
plt.rcParams["figure.figsize"] = (15,7)

### Load images from zarr and compress into max projections

In [2]:
gfp = da.from_zarr('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch1.zarr/')
rfp = da.from_zarr('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch2.zarr/')
gfp = da.sum(gfp, axis = 1, dtype='u2')
rfp = da.sum(rfp, axis = 1, dtype='u2')
masks = da.from_zarr(f'/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch99.zarr/',)# chunks = gfp.chunks)

In [3]:
masks

dask.array<from-zarr, shape=(75, 6048, 6048), dtype=uint16, chunksize=(1, 2160, 2160), chunktype=numpy.ndarray>

## Define features

In [4]:
FEATURES = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "solidity",
  "mean_intensity",
]

In [5]:
%%time

objects = btrack.utils.segmentation_to_objects(
    masks, 
    gfp,
    properties = tuple(FEATURES), 
)

[INFO][2022/08/08 02:40:50 PM] Localizing objects from segmentation...
[INFO][2022/08/08 02:40:50 PM] Found intensity_image data
[INFO][2022/08/08 02:40:50 PM] Calculating weighted centroids using intensity_image
[INFO][2022/08/08 02:47:27 PM] Objects are of type: <class 'dict'>
[INFO][2022/08/08 02:47:27 PM] ...Found 34804 objects in 75 frames.


CPU times: user 19min 11s, sys: 38.1 s, total: 19min 49s
Wall time: 6min 37s


# Parameters to search over

The documents online (https://btrack.readthedocs.io/en/latest/user_guide/configuration.html#tips-and-warnings) also suggest to start with the motion model parameters without using the optimisation step

In [114]:
### these sigma values scale the error associated with each matrix??
P_sigma = [75, 150, 300, 600]
G_sigma = [5, 15, 30, 60]
R_sigma = [1, 5, 15, 30]

Keep note of which variable is which with dict

In [115]:
param_dict_motion_model = {'P_sigma':P_sigma, 
                           'G_sigma':G_sigma,
                           'R_sigma':R_sigma,}

Check that I can iterate over

In [116]:
print(list(itertools.product(*param_dict_motion_model.values())))
print('Number of iterations to cover all params:',len(list(itertools.product(*param_dict_motion_model.values()))))

[(75, 5, 1), (75, 5, 5), (75, 5, 15), (75, 5, 30), (75, 15, 1), (75, 15, 5), (75, 15, 15), (75, 15, 30), (75, 30, 1), (75, 30, 5), (75, 30, 15), (75, 30, 30), (75, 60, 1), (75, 60, 5), (75, 60, 15), (75, 60, 30), (150, 5, 1), (150, 5, 5), (150, 5, 15), (150, 5, 30), (150, 15, 1), (150, 15, 5), (150, 15, 15), (150, 15, 30), (150, 30, 1), (150, 30, 5), (150, 30, 15), (150, 30, 30), (150, 60, 1), (150, 60, 5), (150, 60, 15), (150, 60, 30), (300, 5, 1), (300, 5, 5), (300, 5, 15), (300, 5, 30), (300, 15, 1), (300, 15, 5), (300, 15, 15), (300, 15, 30), (300, 30, 1), (300, 30, 5), (300, 30, 15), (300, 30, 30), (300, 60, 1), (300, 60, 5), (300, 60, 15), (300, 60, 30), (600, 5, 1), (600, 5, 5), (600, 5, 15), (600, 5, 30), (600, 15, 1), (600, 15, 5), (600, 15, 15), (600, 15, 30), (600, 30, 1), (600, 30, 5), (600, 30, 15), (600, 30, 30), (600, 60, 1), (600, 60, 5), (600, 60, 15), (600, 60, 30)]
Number of iterations to cover all params: 64


In [117]:
param_configs = list(itertools.product(*param_dict_motion_model.values()))
stat_dict = {}
for config in tqdm(param_configs):
    ### assign values from config
    P_sigma_i = config[0]
    G_sigma_i = config[1]
    R_sigma_i = config[2]
    ### load config file
    config_fn = '/home/dayn/analysis/BayesianTracker/models/param_test_config.json'
    with open(config_fn,'r+') as jsonfile:
        config_json = json.load(jsonfile)
        ### edit params
        config_json['TrackerConfig']['MotionModel']['P']['sigma'] = P_sigma_i
        config_json['TrackerConfig']['MotionModel']['G']['sigma'] = R_sigma_i
        config_json['TrackerConfig']['MotionModel']['R']['sigma'] = R_sigma_i
        ### dno what this does lol
        jsonfile.seek(0)
        ### save out as temporary config
        json.dump(config_json, jsonfile, indent = 4)
        jsonfile.truncate()  
    
    ### now try tracking with that configuration
    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure_from_file(config_fn)
        tracker.verbose = True
        tracker.max_search_radius = 500
        # use visual features to track
        tracker.features = FEATURES
        # append the objects to be tracked
        tracker.append(objects)
        # set the volume
        tracker.volume=((0, 6048), (0, 6048), (-1e5, 1e5))
        # track them (in interactive mode)
        tracker.track_interactive(step_size=25)
        # generate hypotheses and run the global optimizer
        #tracker.optimize()
        tracks = tracker.tracks
        # store the configuration
        cfg = tracker.configuration
        ## get lazy assessment of tracking performance
        average_track_len = np.mean([len(track) for track in tracks])
        ### store stats in dictionary
        stat_dict[average_track_len] = cfg
        print(config, average_track_len)

  0%|          | 0/64 [00:00<?, ?it/s]

[INFO][2022/08/08 04:52:18 PM] Loaded btrack: /home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/btrack/libs/libtracker.so
[INFO][2022/08/08 04:52:18 PM] btrack (v0.4.5) library imported
[INFO][2022/08/08 04:52:18 PM] Starting BayesianTracker session
[INFO][2022/08/08 04:52:18 PM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/param_test_config.json
[INFO][2022/08/08 04:52:18 PM] Setting max_search_radius -> 500
[INFO][2022/08/08 04:52:18 PM] Setting features -> ['area', 'major_axis_length', 'minor_axis_length', 'orientation', 'solidity', 'mean_intensity']
[INFO][2022/08/08 04:52:18 PM] Objects are of type: <class 'list'>
[INFO][2022/08/08 04:52:18 PM] Setting volume -> ((0, 6048), (0, 6048), (-100000.0, 100000.0))
[INFO][2022/08/08 04:52:18 PM] Starting tracking... 
[INFO][2022/08/08 04:52:19 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 04:52:36 PM]  - Timing (Bayesian updates: 649.70ms, Linking: 12.84ms)
[INFO][2022/08/08 04:52:

(75, 5, 1) 1.9325336055810787


[INFO][2022/08/08 04:52:52 PM] Setting volume -> ((0, 6048), (0, 6048), (-100000.0, 100000.0))
[INFO][2022/08/08 04:52:52 PM] Starting tracking... 
[INFO][2022/08/08 04:52:52 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 04:53:08 PM]  - Timing (Bayesian updates: 557.85ms, Linking: 12.95ms)
[INFO][2022/08/08 04:53:08 PM]  - Probabilities (Link: 0.99726, Lost: 0.99874)
[INFO][2022/08/08 04:53:08 PM]  - Stats (Active: 2757, Lost: 59809, Conflicts resolved: 1041)
[INFO][2022/08/08 04:53:08 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 04:53:20 PM]  - Timing (Bayesian updates: 193.39ms, Linking: 6.93ms)
[INFO][2022/08/08 04:53:20 PM]  - Probabilities (Link: 0.99998, Lost: 1.00000)
[INFO][2022/08/08 04:53:20 PM]  - Stats (Active: 1822, Lost: 120110, Conflicts resolved: 1848)
[INFO][2022/08/08 04:53:20 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 04:53:21 PM]  - Timing (Bayesian updates: 12.83ms, Linking: 1.11ms)
[INFO][2022

(75, 5, 5) 2.3372431265834663


...

(600, 60, 5) 2.5407422765913332


[INFO][2022/08/08 05:20:45 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 05:20:57 PM]  - Timing (Bayesian updates: 466.65ms, Linking: 10.01ms)
[INFO][2022/08/08 05:20:57 PM]  - Probabilities (Link: 0.92513, Lost: 1.00000)
[INFO][2022/08/08 05:20:57 PM]  - Stats (Active: 2396, Lost: 47115, Conflicts resolved: 1966)
[INFO][2022/08/08 05:20:57 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 05:21:07 PM]  - Timing (Bayesian updates: 158.76ms, Linking: 4.80ms)
[INFO][2022/08/08 05:21:07 PM]  - Probabilities (Link: 0.77520, Lost: 1.00000)
[INFO][2022/08/08 05:21:07 PM]  - Stats (Active: 1603, Lost: 96534, Conflicts resolved: 3499)
[INFO][2022/08/08 05:21:07 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 05:21:08 PM]  - Timing (Bayesian updates: 10.70ms, Linking: 1.05ms)
[INFO][2022/08/08 05:21:08 PM]  - Probabilities (Link: 0.99950, Lost: 1.00000)
[INFO][2022/08/08 05:21:08 PM] SUCCESS.
[INFO][2022/08/08 05:21:08 PM]  - Found 1

(600, 60, 15) 3.3305187705017616


[INFO][2022/08/08 05:21:09 PM] Setting volume -> ((0, 6048), (0, 6048), (-100000.0, 100000.0))
[INFO][2022/08/08 05:21:09 PM] Starting tracking... 
[INFO][2022/08/08 05:21:09 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 05:21:21 PM]  - Timing (Bayesian updates: 422.45ms, Linking: 8.76ms)
[INFO][2022/08/08 05:21:21 PM]  - Probabilities (Link: 0.92513, Lost: 1.00000)
[INFO][2022/08/08 05:21:21 PM]  - Stats (Active: 2178, Lost: 40430, Conflicts resolved: 2750)
[INFO][2022/08/08 05:21:21 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 05:21:30 PM]  - Timing (Bayesian updates: 156.75ms, Linking: 5.27ms)
[INFO][2022/08/08 05:21:30 PM]  - Probabilities (Link: 0.58114, Lost: 1.00000)
[INFO][2022/08/08 05:21:30 PM]  - Stats (Active: 1485, Lost: 84046, Conflicts resolved: 5049)
[INFO][2022/08/08 05:21:30 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 05:21:31 PM]  - Timing (Bayesian updates: 9.35ms, Linking: 0.98ms)
[INFO][2022/08

(600, 60, 30) 3.8893867264071686


In [120]:
import pickle

In [121]:
with open('motion_model_params_results_1.pkl', 'wb') as f:
    pickle.dump(stat_dict, f)

# Now with more values

In [128]:
### these sigma values scale the error associated with each matrix??
P_sigma = np.linspace(1, 1000, 10)#[75, 150, 300, 600]
G_sigma = np.linspace(1, 150, 10)#[5, 15, 30, 60]
R_sigma = np.linspace(1, 150, 10)#[1, 5, 15, 30]

Keep note of which variable is which with dict

In [126]:
param_dict_motion_model = {'P_sigma':P_sigma, 
                           'G_sigma':G_sigma,
                           'R_sigma':R_sigma,}

Check that I can iterate over

In [129]:
#print(list(itertools.product(*param_dict_motion_model.values())))
print('Number of iterations to cover all params:',len(list(itertools.product(*param_dict_motion_model.values()))))

Number of iterations to cover all params: 512


In [130]:
param_configs = list(itertools.product(*param_dict_motion_model.values()))
stat_dict = {}
for config in tqdm(param_configs):
    try:
        ### assign values from config
        P_sigma_i = config[0]
        G_sigma_i = config[1]
        R_sigma_i = config[2]
        ### load config file
        config_fn = '/home/dayn/analysis/BayesianTracker/models/param_test_config.json'
        with open(config_fn,'r+') as jsonfile:
            config_json = json.load(jsonfile)
            ### edit params
            config_json['TrackerConfig']['MotionModel']['P']['sigma'] = P_sigma_i
            config_json['TrackerConfig']['MotionModel']['G']['sigma'] = R_sigma_i
            config_json['TrackerConfig']['MotionModel']['R']['sigma'] = R_sigma_i
            ### dno what this does lol
            jsonfile.seek(0)
            ### save out as temporary config
            json.dump(config_json, jsonfile, indent = 4)
            jsonfile.truncate()  

        ### now try tracking with that configuration
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:
            # configure the tracker using a config file
            tracker.configure_from_file(config_fn)
            tracker.verbose = True
            tracker.max_search_radius = 500
            # use visual features to track
            tracker.features = FEATURES
            # append the objects to be tracked
            tracker.append(objects)
            # set the volume
            tracker.volume=((0, 6048), (0, 6048), (-1e5, 1e5))
            # track them (in interactive mode)
            tracker.track_interactive(step_size=25)
            # generate hypotheses and run the global optimizer
            #tracker.optimize()
            tracks = tracker.tracks
            # store the configuration
            cfg = tracker.configuration
            ## get lazy assessment of tracking performance
            average_track_len = np.mean([len(track) for track in tracks])
            ### store stats in dictionary
            stat_dict[average_track_len] = cfg
            print(config, average_track_len)
        with open('motion_model_params_results_update.pkl', 'wb') as f:
            pickle.dump(stat_dict, f)
    except Exception as e: print(e)

  0%|          | 0/512 [00:00<?, ?it/s]

[INFO][2022/08/08 05:24:13 PM] Loaded btrack: /home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/btrack/libs/libtracker.so
[INFO][2022/08/08 05:24:13 PM] btrack (v0.4.5) library imported
[INFO][2022/08/08 05:24:13 PM] Starting BayesianTracker session
[INFO][2022/08/08 05:24:13 PM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/param_test_config.json
[INFO][2022/08/08 05:24:13 PM] Setting max_search_radius -> 500
[INFO][2022/08/08 05:24:13 PM] Setting features -> ['area', 'major_axis_length', 'minor_axis_length', 'orientation', 'solidity', 'mean_intensity']
[INFO][2022/08/08 05:24:13 PM] Objects are of type: <class 'list'>
[INFO][2022/08/08 05:24:14 PM] Setting volume -> ((0, 6048), (0, 6048), (-100000.0, 100000.0))
[INFO][2022/08/08 05:24:14 PM] Starting tracking... 
[INFO][2022/08/08 05:24:14 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 05:24:30 PM]  - Timing (Bayesian updates: 613.25ms, Linking: 14.24ms)
[INFO][2022/08/08 05:24:

(1.0, 1.0, 1.0) 1.6896314157767827


[INFO][2022/08/08 05:24:46 PM] Setting volume -> ((0, 6048), (0, 6048), (-100000.0, 100000.0))
[INFO][2022/08/08 05:24:46 PM] Starting tracking... 
[INFO][2022/08/08 05:24:46 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 05:24:59 PM]  - Timing (Bayesian updates: 493.53ms, Linking: 11.55ms)
[INFO][2022/08/08 05:24:59 PM]  - Probabilities (Link: 0.81291, Lost: 1.00000)
[INFO][2022/08/08 05:24:59 PM]  - Stats (Active: 2445, Lost: 47947, Conflicts resolved: 2826)
[INFO][2022/08/08 05:24:59 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 05:25:10 PM]  - Timing (Bayesian updates: 171.34ms, Linking: 5.90ms)
[INFO][2022/08/08 05:25:10 PM]  - Probabilities (Link: 0.52848, Lost: 1.00000)
[INFO][2022/08/08 05:25:10 PM]  - Stats (Active: 1642, Lost: 98670, Conflicts resolved: 4973)
[INFO][2022/08/08 05:25:10 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 05:25:11 PM]  - Timing (Bayesian updates: 10.85ms, Linking: 1.05ms)
[INFO][2022/

(1.0, 1.0, 22.285714285714285) 3.6980283425754776


...

(1000.0, 150.0, 107.42857142857142) 1.988425171177046


[INFO][2022/08/08 08:55:51 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 08:56:04 PM]  - Timing (Bayesian updates: 465.46ms, Linking: 9.35ms)
[INFO][2022/08/08 08:56:04 PM]  - Probabilities (Link: 0.94744, Lost: 1.00000)
[INFO][2022/08/08 08:56:04 PM]  - Stats (Active: 2323, Lost: 49592, Conflicts resolved: 18)
[INFO][2022/08/08 08:56:04 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 08:56:15 PM]  - Timing (Bayesian updates: 161.06ms, Linking: 4.61ms)
[INFO][2022/08/08 08:56:15 PM]  - Probabilities (Link: 0.89725, Lost: 1.00000)
[INFO][2022/08/08 08:56:15 PM]  - Stats (Active: 1524, Lost: 102090, Conflicts resolved: 51)
[INFO][2022/08/08 08:56:15 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 08:56:16 PM]  - Timing (Bayesian updates: 10.18ms, Linking: 0.91ms)
[INFO][2022/08/08 08:56:16 PM]  - Probabilities (Link: 0.94643, Lost: 1.00000)
[INFO][2022/08/08 08:56:16 PM] SUCCESS.
[INFO][2022/08/08 08:56:16 PM]  - Found 19833

(1000.0, 150.0, 128.71428571428572) 1.754853022739878


[INFO][2022/08/08 08:56:17 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2022/08/08 08:56:29 PM]  - Timing (Bayesian updates: 467.35ms, Linking: 9.13ms)
[INFO][2022/08/08 08:56:29 PM]  - Probabilities (Link: 0.94744, Lost: 1.00000)
[INFO][2022/08/08 08:56:29 PM]  - Stats (Active: 2304, Lost: 49076, Conflicts resolved: 18)
[INFO][2022/08/08 08:56:29 PM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2022/08/08 08:56:39 PM]  - Timing (Bayesian updates: 161.37ms, Linking: 4.60ms)
[INFO][2022/08/08 08:56:39 PM]  - Probabilities (Link: 0.89725, Lost: 1.00000)
[INFO][2022/08/08 08:56:39 PM]  - Stats (Active: 1519, Lost: 101153, Conflicts resolved: 51)
[INFO][2022/08/08 08:56:39 PM] Tracking objects in frames 50 to 74 (of 75)...
[INFO][2022/08/08 08:56:40 PM]  - Timing (Bayesian updates: 10.01ms, Linking: 0.91ms)
[INFO][2022/08/08 08:56:40 PM]  - Probabilities (Link: 0.95142, Lost: 1.00000)
[INFO][2022/08/08 08:56:40 PM] SUCCESS.
[INFO][2022/08/08 08:56:40 PM]  - Found 19653

(1000.0, 150.0, 150.0) 1.7709255584389152


In [131]:
with open('motion_model_params_results_2.pkl', 'wb') as f:
    pickle.dump(stat_dict, f)

In [134]:
len(stat_dict)

64

In [133]:
len(stat_dict)

64

In [135]:
means = []
for i in stat_dict:
    means.append(i)

In [136]:
max(means)-min(means)

2.490168993708532

In [137]:
max(means)

3.7579357222992145

In [138]:
stat_dict[3.7579357222992145]

TrackerConfig(name='Default', version='0.4.5', verbose=True, motion_model=MotionModel(measurements=3, states=6, A=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]), H=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]]), P=array([[ 42.91428571,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  42.91428571,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  42.91428571,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , 429.14285714,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        429.14285714,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.